![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Proyecto final - Data Analytics y Business Intelligence

Recibimos dos datasets:

1. `renfe.csv`: Información de búsquedas de billetes que se hicieron en la página de Renfe.
2. `coordenadas_ciudades.csv`: Latitud y longitud de provincias españolas.

Queremos usar estos datasets para un modelo de Machine Learning que utilizaremos para predecir los precios de los billetes. Y, para ello, necesitamos limpiar, explorar y pre-procesar el dataset.

## Reglas de juego

1. El proyecto se debe entregar en grupos de dos o individualmente. 
2. Cada respuesta correcta suma un punto.
3. La calificación final consistirá en la suma de todos los puntos obtenidos sobre el total de puntos posibles.


## Diccionario de datos

Esta es la información provista:

### `renfe.csv`
- `FECHA_CONSULTA`: Fecha en la que se consultó la página.
- `FECHA_INICIO`: Fecha de inicio del trayecto.
- `FECHA_FIN`: Fecha de finalización del trayecto.
- `CIUDAD_ORIGEN`: Ciudad de origen del trayecto.
- `CIUDAD_DESTINO`: Ciudad destino del trayecto.
- `TIPO_TREN`: Tipo de tren.
- `TIPO_TARIFA`: Tipo de tarifa del billete.
- `CLASE`: Clase del asiento seleccionado.
- `PRECIO`: Precio del tren seleccionado.

### `coordenadas_ciudades.csv`
- `ciudad`: Nombre de la ciudad.
- `latitud`: Coordenada de latitud de la ciudad.
- `longitud`: Coordenada de longitud de la ciudad.

## Importar librerías

In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
import folium

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## P0: Lee el dataset `renfe.csv`

In [3]:
df = pd.read_csv("data/renfe.csv", sep=";")

## P1: Visualiza las primeras y las últimas filas del dataset

In [ ]:
df.head(10)


In [ ]:
df.tail(10)

In [ ]:
#cojo una muestra aleatoria también para ver si ya puedo ir encontrando algo
df.sample(n=10,random_state=42)

## P2: ¿Cuantas filas y columnas tiene el dataset?

In [ ]:
#9 columnas y 383568 filas
df.shape

## P3: Cambia los nombres de todas las columnas a minúsculas

In [8]:
df.columns = [x.lower() for x in df.columns]

In [ ]:
df.columns

## P4: Muestra los tipos de datos de cada columna

In [ ]:

df.info(memory_usage='deep')

## P5: Cambia los tipos de datos que creas que creas incorrectos, por los tipos adecuados

In [ ]:
#voy a cambiar a tipo fecha las fechas
columnas = ['fecha_consulta', 'fecha_inicio', 'fecha_fin']
for x in columnas:
    df[x] = pd.to_datetime(df[x])

df.head()
df.info(memory_usage='deep')
#veo que se han reducido 65,8 MB


## P6: Filas duplicadas

### P6.1: ¿Cuántas filas duplicadas tiene el dataset?

In [ ]:
#esto me da el número de las filas que están completamente duplicadas - 33
df.duplicated().sum()

In [ ]:
#para verlas:
df.loc[df.duplicated(keep=False)]
#como keep=False, localiza todos los que se han encontrado más de una vez, es decir, marca como duplicado todas las apariciones
#bastante probable que haya 33 pares de filas duplicadas porque el total es 66

### P6.2: Quita las filas duplicadas

In [17]:
df = df.drop_duplicates() #se queda con la primera aparición por defecto

In [ ]:
df.duplicated().sum()  # ahora debería dar 0 

## P7: Valores nulos y análisis de `precio`

### P7.1: ¿Que porcentaje de valores nulos hay por cada columna?

In [ ]:
df.isnull().mean() * 100

In [20]:
#tenemos nulos en tipo_tarifa, clase y precio 

In [ ]:
#me quiero asegurar también que las otras columnas categóricas no tengan valores raros
df.ciudad_destino.value_counts(dropna=False)

In [ ]:
df.ciudad_origen.value_counts(dropna=False)

In [ ]:
df.tipo_tren.value_counts(dropna=False)

In [ ]:
df.tipo_tarifa.value_counts(dropna=False)

In [ ]:
df.clase.value_counts(dropna=False)

### P7.2: ¿Cual es el mínimo, percentiles importantes (25%, 50%, 75%) y el máximo de `precio`?

In [ ]:
df['precio'].describe(percentiles = [0.01,0.05,0.25, 0.75, 0.9,0.95])

#el mínimo es 0.0, el 25% es 41,2€, el 50% es 58,15€, el 75% es 76,3€ y el máximo 342,8€ 
#viendo la media y la mediana podemos ver que es una distribución mayoritariamente simétrica, un pelín pelín desplazada a la derecha si es que se puede considerar como desplazada
#ya podemos ir viendo los atípicos que tenemos, por ejem el valor que nos da el máximo

In [ ]:
#quiero ver mejor la distribución
px.histogram(df, x='precio', nbins=50, title='Distribución de Precios')
#vemos el pico - hay más billetes alrededor de los 45-55 euros. Y los atípicos a la derecha, a partir de 175 €  aprox
#muy importante visualizar nuestra variable objetivo!
#como tiene atípicos, luego quizá necesitaríamos hacer transformaciones


### P7.3: ¿Hay algo raro en el valor mínimo de `precio`? Quita las filas con ese valor del dataset

In [28]:
df = df[df['precio'] > 0]

In [ ]:
(df['precio'] == 0_0).sum()

### P7.4: Reemplaza los valores nulos en `precio` por la media de esa columna

In [30]:
media = df.precio.mean()  #trazabilidad de lo que guardamos, sobretodo en datos numéricos
df = df.fillna({'precio': media})

### P7.5: Quita las filas donde `clase` o `tipo_tarifa` sean nulos

In [31]:
#df = df[~df['clase'].isna() & ~df['tipo_tarifa'].isna()]
df = df.dropna(subset=['clase', 'tipo_tarifa'])

## P8: Tiempo de viaje

### P8.1: Calcula el tiempo de viaje en minutos (fecha_fin - fecha_inicio)

In [ ]:
df['tiempo_viaje'] = df['fecha_fin'] - df['fecha_inicio']
df.tiempo_viaje.head()
df.tiempo_viaje.info()
#tiene un formato timedelta64 (0 days 02:38:00) y no sale en minutos

In [ ]:
#lo convierto a minutos
df['tiempo_viaje'] = df['tiempo_viaje'].dt.total_seconds() / 60
df.tiempo_viaje.head()

### P8.2: Haz un histograma de la variable que acabas de crear (`tiempo_de_viaje`)

In [ ]:
px.histogram(df,x='tiempo_viaje',nbins=50, title='Distribución del Tiempo de Viaje')

## P9: Extrae el día, el nombre del día, el mes y la hora de `fecha_inicio`

In [ ]:
#entiendo que tenemos que crear columnas nuevas
#formato actual: 2019-06-28 20:36:00
#nueva columna con el día
df['fecha_inicio_dia'] = df['fecha_inicio'].dt.day
df['fecha_inicio_dia'].head()

In [ ]:
#nueva columna con el mes
df['fecha_inicio_mes'] = df['fecha_inicio'].dt.month
df['fecha_inicio_mes'].head()

In [ ]:
#nueva columna con la hora
df['fecha_inicio_hora'] = df['fecha_inicio'].dt.hour
df['fecha_inicio_hora'].head()

In [ ]:
#nueva columna con el nombre del día
#según la documentación de pandas, si ejecutamos locale -a en la terminal podemos encontrar nuestro locale lenguage code, me salían varios, pero este parece que pilla bien las tiles, este no (es_ES.utf8)
df['fecha_inicio_nombre_dia'] = df['fecha_inicio'].dt.day_name(locale='es_ES')
df['fecha_inicio_nombre_dia'].unique()

In [ ]:
df[['fecha_inicio', 'fecha_inicio_dia', 'fecha_inicio_nombre_dia', 'fecha_inicio_mes', 'fecha_inicio_hora']].head()

## P10: Quita las columnas `fecha_consulta`, `fecha_inicio` y `fecha_fin` del dataset

In [40]:
df = df.drop(columns=['fecha_consulta', 'fecha_inicio', 'fecha_fin'])

In [41]:
#no estaba segura si debería hacer un reset_index después de todos los cambios, pero no hace falta con el merge, lo dejo así 

## P11: Lee el dataset `coordenadas_ciudades.csv` y únelo con al dataset que has procesado hasta ahora (utiliza `ciudad_destino` para el `join`)

In [42]:
df_coordenadas = pd.read_csv("data/coordenadas_ciudades.csv")

In [ ]:
df_coordenadas.head(5)

In [44]:
#hacemos merge
df = df.merge(df_coordenadas, left_on='ciudad_destino',right_on='ciudad',how='left')

## P12: Gráfica en un mapa el precio medio por ciudad de destino

In [ ]:
df.columns

In [46]:
#para graficar el precio medio por ciudad, primero tenemos que sacar el precio medio 

df_precio_medio = df.groupby('ciudad_destino').agg({
    'precio': 'mean',
    'latitud': 'first',   # toma la primera latitud (todas son iguales por ciudad)
    'longitud': 'first'
}).reset_index()


In [ ]:
print(df_precio_medio.columns)


In [ ]:
#nunca había buscado las coordenadas de mi casa hasta ahora
#he cogido el mapa que se llama CartoDB.VoyagerLabelsUnder
attr = (
    'Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a> | Map style: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>)'
)
tiles = "https://{s}.basemaps.cartocdn.com/rastertiles/voyager_labels_under/{z}/{x}/{y}{r}.png"
precio_medio_mapa = folium.Map(location=[40.442218772534616, -3.702914307732981], tiles=tiles, attr=attr, zoom_start=7)

for idx, row in df_precio_medio.iterrows():
    folium.Circle(
        location=(row["latitud"], row["longitud"]),
        radius=row["precio"] * 800,  #círculos más grandes
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        color="red",
        weight=3,
        tooltip=f"""
        <ul>
            <li>Ciudad: {row["ciudad_destino"]}</li>
            <li>Precio medio del billete: {row["precio"]:.2f}€</li>
        <ul>
        """
    ).add_to(precio_medio_mapa)
precio_medio_mapa

## P13: Haz una tabla de correlación, ¿hay variables númericas correladas con el precio?

In [ ]:

columnas_correlacion = df.select_dtypes(include='number').columns.tolist()
columnas_correlacion = [col for col in columnas_correlacion if col not in ['latitud', 'longitud']]

px.imshow(
    df[columnas_correlacion].corr(),
    height=700,
    color_continuous_midpoint=0,
    range_color=[-1, 1],
    color_continuous_scale='viridis',  
    text_auto=True,
    title='Matriz de Correlación'
)

#las correlaciones van de -1 a +1:
#a mayores correlaciones, debería pasar que si una variable sube, la otra también (máx 1)
#a menores correlaciones, debería pasar que una suba y otra baje (máx -1)
#correlación cercana a 0: no hay relación LINEAL entre variables

#no hay muchas correlaciones positivas 
#podemos ver que la correlación más negativa es el tiempo del viaje.Que puede ser perfectamente, porque si el trayecto del viaje es muy largo como puede pasar en trenes regionales, el precio es más barato
#habría que seguir investigando la correlación con otras variables categóricas como el tipo_tren o la clase

## P14: Relación entre variables del dataset y `precio`

### P14.1: Haz un scatter plot de precio vs. tiempo de viaje

In [ ]:
px.scatter(
    df,
    x='tiempo_viaje',
    y='precio',
    color='tipo_tren',  # colorea por tipo de tren
    title='Relación entre Precio y Tiempo de Viaje',
    labels={
        'tiempo_viaje': 'Tiempo de viaje (minutos)',
        'precio': 'Precio (€)',
        'tipo_tren': 'Tipo de Tren'
    },
    opacity=0.6,
    hover_data=['ciudad_origen', 'ciudad_destino']  # info al pasar el mouse
)

In [52]:
#he puesto por color el tipo de tren por si veo algún patrón - la mayor parte de los trayectos son cortos y de AVE y precios más variados 
#pero vamos que como son tantos puntos se ve regulinchi, ha tardado solo 6 segundos en ejecutarse, pero ya me estaba asustando

In [ ]:
#si lo hago con una muestra aleatoria con menos información, a ver qué pasa 
df_muestra = df.sample(n=5000, random_state=42)

px.scatter(
    df_muestra,
    x='tiempo_viaje',
    y='precio',
    color='tipo_tren',
    title='Relación entre Precio y Tiempo de Viaje',
    opacity=0.6
)

#lo veo parecido, no veo una relación lineal clara, ya nos lo estaba diciendo la matriz de correlación. Sí que puede ser que haya ciertos clusters, los de trayectos cortos con precios altos, trayectos largos con precios bajos


### P14.2: Haz un boxplot de precio vs. dia de la semana

In [ ]:
df.columns

In [ ]:
px.violin(df, x= 'fecha_inicio_nombre_dia', y='precio', title= 'Relación entre Precio y Día de la semana', color = "fecha_inicio_nombre_dia", hover_data = 'precio')

In [56]:
#no veo que haya mucha diferencia, las distribuciones están en niveles muy parecidos, quizá un pelín más altos el domingo, viernes y lunes por ser finde semana
#y hay atípicos todos los días puta renfe

### P14.3: Gráfica el precio medio por día de la semana

In [57]:
precio_por_dia = df.groupby('fecha_inicio_nombre_dia')['precio'].mean()


In [ ]:
df.head()

In [59]:
precio_por_dia= precio_por_dia.reset_index()

In [ ]:
px.bar(
    precio_por_dia,  
    x='fecha_inicio_nombre_dia',
    y='precio',
    title='Precio Medio por Día de la Semana',
    labels={
        'fecha_inicio_nombre_dia': 'Día de la semana',
        'precio': 'Precio medio (€)'
    },
    color='precio',  # Color basado en el valor del precio
    color_continuous_scale='Viridis'  # Puedes cambiar la paleta
)


## P15: Crea un nuevo dataframe donge apliques *one-hot-encoding* a las variables categoricas

In [ ]:
df.head()

In [63]:
#las columnas a convertir serán: la de las ciudades, tipo_tren, tipo_tarifa, clase y fecha_inicio_nombre_dia 

In [64]:
#Label Encoding es solo para la variable objetivo que queremos predecir
#Para aplicar one-hot-encoding puedo hacerlo desde pandas con get_dummies o con una clase desde Scikit-Learn
#Lo hago con get_dummies

In [65]:
columnas_categoricas = ['tipo_tren', 'tipo_tarifa', 'clase', 'fecha_inicio_nombre_dia', 'ciudad', "ciudad_origen", "ciudad_destino"]

In [66]:
df_enc = pd.get_dummies(df,columns=columnas_categoricas,drop_first=True,dtype=int)
#he forzado a que sean enteros
#drop_first = True para reducir el nº de columnas

In [ ]:
df_enc.head()